<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [1]:
!git clone 'https://github.com/sumandutta8877/Data_DA.git'    # Run this Line for only one time

Cloning into 'Data_DA'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [607]:
import pandas as pd
import numpy as np
import math
df_train = pd.read_csv('/content/Data_DA/Data/Train_data.csv')
df_test = pd.read_csv('/content/Data_DA/Data/Test_data.csv')
df_sample = pd.read_csv('/content/Data_DA/Data/Sample_submission.csv')
#print(df_train.head)
#print(df_test.head)
#print(df_sample.head)
df1 = df_train            # df1 is the training dataframe
df2 = df_test             # df2 is the test dataframe

#Pre-Processing

In [608]:
df1['mode'] = df1['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df1['explicit'] = df1['explicit'].apply(lambda x: 0.5 if x == 'No' else 1.2)
df1['id'] = 1
df2['mode'] = df2['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df2['explicit'] = df2['explicit'].apply(lambda x: 0.5 if x == 'No' else 1.2)
df2['id'] = 1
df1.loc[(df1['popularity'] == 'very low'),'ratings'] = 1
df1.loc[(df1['popularity'] == 'low'),'ratings'] = 2
df1.loc[(df1['popularity'] == 'average'),'ratings'] = 3
df1.loc[(df1['popularity'] == 'high'),'ratings'] = 4
df1.loc[(df1['popularity'] == 'very high'),'ratings'] = 5
M_vt = df1['ratings'].to_numpy()
del df1['ratings']
del df1['popularity']
del df1['year']
del df2['year']
df = df1['id']
df['temp1'] = df1['release_date'].str[0:2].astype(int)
df['temp2'] = df1['release_date'].str[3:5].astype(int)
df['temp3'] = df1['release_date'].str[6:10].astype(int)
df1['loudness'] = -1.0*df1['loudness']
df1['release_date'] = 737900 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today
df = df2['id']
df['temp1'] = df2['release_date'].str[0:2].astype(int)
df['temp2'] = df2['release_date'].str[3:5].astype(int)
df['temp3'] = df2['release_date'].str[6:10].astype(int)
df2['loudness'] = -1.0*df2['loudness']
df2['release_date'] = 737900 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#Tuning Parameters

In [609]:
del df1['instrumentalness']
del df2['instrumentalness']
del df1['key']
del df2['key']
del df1['valence']
del df2['valence']
del df1['liveness']
del df2['liveness']
del df1['tempo']
del df2['tempo']
#del df1['explicit']
#del df2['explicit']
#del df1['release_date']
#del df2['release_date']
#del df1['mode']
#del df2['mode']
#del df1['acousticness']
#del df2['acousticness']
#df1['speechinesss0.5'] = np.sqrt(df1['speechiness'])
#df2['speechiness0.5'] = np.sqrt(df2['speechiness'])
#df1['speechiness2'] = df1['speechiness']*df1['speechiness']
#df2['speechiness2'] = df2['speechiness']*df2['speechiness']
#df1['acousticness3'] = df1['acousticness']*df1['acousticness']*df1['acousticness']
#df2['acousticness3'] = df2['acousticness']*df2['acousticness']*df2['acousticness']
df1['acousticness0.5'] = np.sqrt(df1['acousticness'])
df2['acousticness0.5'] = np.sqrt(df2['acousticness'])
df1['loudness'][4260] = -1.0*df1['loudness'][4260]
df1['loudness0.5'] = np.sqrt(df1['loudness'])
df2['loudness0.5'] = np.sqrt(df2['loudness'])
df1['loudness2'] = df1['loudness']*df1['loudness']
df2['loudness2'] = df2['loudness']*df2['loudness']
df1['loudness3'] = df1['loudness']*df1['loudness']*df1['loudness']
df2['loudness3'] = df2['loudness']*df2['loudness']*df2['loudness']
df1['duration-min0.5'] = np.sqrt(df1['duration-min'])
df2['duration-min0.5'] = np.sqrt(df2['duration-min'])
df1['duration-min2'] = df1['duration-min']*df1['duration-min']
df2['duration-min2'] = df2['duration-min']*df2['duration-min']
df1['duration-min3'] = df1['duration-min']*df1['duration-min']*df1['duration-min']
df2['duration-min3'] = df2['duration-min']*df2['duration-min']*df2['duration-min']
df1['danceability2'] = df1['danceability']*df1['danceability']
df2['danceability2'] = df2['danceability']*df2['danceability']
df1['danceability3'] = df1['danceability']*df1['danceability']*df1['danceability']
df2['danceability3'] = df2['danceability']*df2['danceability']*df2['danceability']
df1['energy2'] = df1['energy']*df1['energy']
df2['energy2'] = df2['energy']*df2['energy']
df1['energy3'] = df1['energy']*df1['energy']*df1['energy']
df2['energy3'] = df2['energy']*df2['energy']*df2['energy']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#Calculations

In [610]:
M_tr = df1.to_numpy()
M_ts = df2.to_numpy()
W = np.matmul((np.matmul(np.linalg.inv(np.matmul(np.transpose(M_tr),M_tr)),np.transpose(M_tr))),M_vt)
Y_output = np.matmul(M_ts,W)
A_test = np.matmul(M_tr,W)

Error = 0.0
mean = 0.0
mean_t = 0.0
x = 0.56275
mis_match = 0
a = 1.15
b = 0.97
c = 1.1
d = 0.75

for i in range(len(A_test)):
  if A_test[i]<=1.0+x*a:
    A_test[i]=1.0
  elif A_test[i]<=2.0+x*b:
    A_test[i]=2.0
  elif A_test[i]<=3.0+x*c:
    A_test[i]=3.0
  elif A_test[i]<=4.0+x*d:
    A_test[i]=4.0
  else:
    A_test[i]=5.0
  mean_t = mean_t + A_test[i]
mean_t = mean_t/(1.0*len(A_test))
for i in range(len(M_vt)):
  Error = Error + abs(M_vt[i]-math.ceil(A_test[i]))/5.0
  mean = mean + M_vt[i]
  if (M_vt[i]-math.ceil(A_test[i])) !=0:
    mis_match = mis_match+1
mis_match = mis_match/(1.0*len(M_vt))
mean = mean/(1.0*len(M_vt))
Error = Error/(1.0*len(M_vt))
Accuracy = 100.0*(1.0 - Error)
for i in range(len(Y_output)):
  if Y_output[i]<=1.0+x*a:
    Y_output[i]=1.0
  elif Y_output[i]<=2.0+x*b:
    Y_output[i]=2.0
  elif Y_output[i]<=3.0+x*c:
    Y_output[i]=3.0
  elif Y_output[i]<=4.0+x*d:
    Y_output[i]=4.0
  else:
    Y_output[i]=5.0
df = pd.DataFrame(Y_output, columns = ['ratings'])
df.loc[(df['ratings'] == 1.0),'popularity'] = 'very low'
df.loc[(df['ratings'] == 2.0),'popularity'] = 'low'
df.loc[(df['ratings'] == 3.0),'popularity'] = 'average'
df.loc[(df['ratings'] == 4.0),'popularity'] = 'high'
df.loc[(df['ratings'] == 5.0),'popularity'] = 'very high'
del df['ratings']

#Print Output

In [611]:
df_id=pd.read_csv(r'/content/Data_DA/Data/Test_data.csv')
data = [df_id['id'],df['popularity']]
headers = ['id','popularity']
df_output = pd.concat(data, axis=1, keys=headers)
df_output.to_csv(r'/content/Data_DA/Data/Sample_output.csv',index=False)
print('Accuracy on train data =',Accuracy,'%')
print('Match percentage on train data =',100.0*(1-mis_match),'%')

Accuracy on train data = 88.28494315858234 %
Match percentage on train data = 51.917886644311764 %


In [612]:
print(W)
print(mean,mean_t)

[ 1.91153030e+00 -1.81638276e+00 -9.26501243e-01 -1.10167195e+00
  5.09347120e-01 -2.40929773e-01  8.36576892e-03 -5.94771859e-05
 -5.66741234e-01 -6.76519709e-01  1.65585840e+00  3.50784965e-01
  7.94671468e-03 -9.58061053e-05  2.23159635e+00  1.42278734e-02
 -1.20588799e-04  2.09381285e+00 -1.26829795e+00  2.28901480e+00
 -2.12588152e+00]
2.491453340966713 2.391019874049235
